# Libraries

In [ ]:
#pip install pandas

In [ ]:
#pip install PyPDF2

In [ ]:
#pip install pdfplumber

In [ ]:
#pip install Pillow

# Extract Text

In [130]:
import os
import pandas as pd
import PyPDF2
import re

def pdf_extract(folder):
    data = []
    for filename in os.listdir(folder):
        if filename.endswith(".pdf"):
            file_path = os.path.join(folder, filename)
            with open(file_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                extracted_text = ''.join(page.extract_text() or '' for page in pdf_reader.pages)
                extracted_text = re.sub(r'\s+', ' ', extracted_text).strip()  # Normalize whitespace

                # Patterns to remove
                patterns = [
                    r'journal of orthopaedic & sports physical therapy',
                    r'\b(volume|number)\s+\d+',
                    r'\b(january|february|march|april|may|june|july|august|september|october|november|december)\s+\d{4}',
                    r'\|',
                    r'\[\s*musculoskeletal imaging\s*\]',
                    r'^\s*\d+\s+'
                ]
                # Apply patterns
                for pattern in patterns:
                    extracted_text = re.sub(pattern, '', extracted_text, flags=re.IGNORECASE)

                # Remove specific characters or patterns
                extracted_text = extracted_text.replace(" - ", "")

                # Append result to list
                data.append({"file_name": filename, "extracted_text": extracted_text})

    # Create DataFrame from list
    return pd.DataFrame(data)


In [ ]:
# Example usage
# folder_path = "sample_data"
# text_df = pdf_extract(folder_path)
# print(text_df.loc[0, 'extracted_text'])

# Extract images

In [85]:
import os
import base64
import pandas as pd
import pdfplumber
import io
from PIL import Image

def extract_images(folder):
    # Initialize a list to store dictionaries for eventual DataFrame creation
    data_list = []

    # Iterate over every file in the specified folder
    for filename in os.listdir(folder):
        if filename.lower().endswith(".pdf"):
            file_path = os.path.join(folder, filename)

            # Initialize a dictionary to hold the base64 encoded images
            images_dict = {}
            image_counter = 1

            # Open the PDF file
            with pdfplumber.open(file_path) as pdf:
                # Extract images from each page
                for page in pdf.pages:
                    if 'images' in page.objects:
                        for img_dict in page.images:
                            # Get the image object using its object ID within the PDF
                            im_obj = pdf.extracted_objects[img_dict['object_id']]
                            # Access the raw image data
                            if im_obj['type'] == 'image':
                                # Convert the raw image data into a PIL Image
                                im = Image.open(io.BytesIO(im_obj['data']))
                                # Convert the image to bytes and then encode in base64
                                buffered = io.BytesIO()
                                im.save(buffered, format="JPEG")
                                img_str = base64.b64encode(buffered.getvalue()).decode('utf-8')
                                # Store the image in dictionary with a unique key
                                images_dict[f'image_{image_counter}'] = img_str
                                image_counter += 1

            # Append the data dictionary to the list
            data_list.append({
                'file_name': filename,
                'extracted_images': images_dict
            })

    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data_list)

    # Return the DataFrame
    return df

In [86]:
# Example use
# image_df = extract_images('sample_data')